In [11]:
import chess
import chess.engine
import chess.pgn

In [12]:
# Paths to your engines and Maia models
stockfish_path = "/opt/homebrew/bin/stockfish"
maia_model_paths = {
    1100: "maia_weights/maia-1100.pb.gz",
    1200: "maia_weights/maia-1200.pb.gz",
    1300: "maia_weights/maia-1300.pb.gz",
    1400: "maia_weights/maia-1400.pb.gz",
    1500: "maia_weights/maia-1500.pb.gz",
    1600: "maia_weights/maia-1600.pb.gz",
    1700: "maia_weights/maia-1700.pb.gz",
    1800: "maia_weights/maia-1800.pb.gz",
    1900: "maia_weights/maia-1900.pb.gz",
}


In [13]:
# Initialize Stockfish engine
stockfish_engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

# Initialize Maia engines
maia_engines = {elo: chess.engine.SimpleEngine.popen_uci(["lc0", f"--weights={path}"]) for elo, path in maia_model_paths.items()}

<UciProtocol (pid=61201)>: stderr >>        _
<UciProtocol (pid=61201)>: stderr >> |   _ | |
<UciProtocol (pid=61201)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=61202)>: stderr >>        _
<UciProtocol (pid=61202)>: stderr >> |   _ | |
<UciProtocol (pid=61202)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=61203)>: stderr >>        _
<UciProtocol (pid=61203)>: stderr >> |   _ | |
<UciProtocol (pid=61203)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=61204)>: stderr >>        _
<UciProtocol (pid=61204)>: stderr >> |   _ | |
<UciProtocol (pid=61204)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=61205)>: stderr >>        _
<UciProtocol (pid=61205)>: stderr >> |   _ | |
<UciProtocol (pid=61205)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=61206)>: stderr >>        _
<UciProtocol (pid=61206)>: stderr >> |   _ | |
<UciProtocol (pid=61206)>: 

In [14]:

def load_games_from_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

In [15]:

def analyze_with_engines(game, stockfish_engine, maia_engines):
    board = game.board()
    blunders = []

    for move in game.mainline_moves():
        board.push(move)

        # Analyze with Stockfish
        stockfish_info = stockfish_engine.analyse(board, chess.engine.Limit(time=0.1))
        stockfish_eval = stockfish_info["score"].relative.score(mate_score=10000) / 100.0

        # Define blunder threshold
        if abs(stockfish_eval) > 1.5:
            maia_suggestions = {}
            for elo, maia_engine in maia_engines.items():
                maia_info = maia_engine.analyse(board, chess.engine.Limit(nodes=1))
                maia_eval = maia_info["score"].relative.score(mate_score=10000) / 100.0
                maia_suggestions[elo] = (maia_info["pv"][0], maia_eval)

            blunders.append((board.fen(), move, stockfish_eval, maia_suggestions))

    return blunders

In [16]:

def output_blunder_analysis(blunders):
    for fen, move, stockfish_eval, maia_suggestions in blunders:
        print(f"Position: {fen}")
        print(f"Blunder: {move}")
        print(f"Stockfish Evaluation: {stockfish_eval}")
        for elo, (best_move, maia_eval) in maia_suggestions.items():
            print(f"Maia {elo} Evaluation: {maia_eval}, Suggested Move: {best_move}")
        print()

In [17]:

# Load games from PGN file
pgn_file_path = "analysis_pgns/0KNdpNQd---1910 Lasker vs. Schlechter.pgn"
all_games = load_games_from_pgn(pgn_file_path)

# Analyze games
all_blunders = []
for game in all_games:
    blunders = analyze_with_engines(game, stockfish_engine, maia_engines)
    all_blunders.append((game.headers["Event"], blunders))

# Output analysis
for event, blunder_list in all_blunders:
    print(f"Event: {event}")
    output_blunder_analysis(blunder_list)

# Cleanup
stockfish_engine.quit()
for maia_engine in maia_engines.values():
    maia_engine.quit()

Event: World Championship Match 1910
Position: 7r/2R5/3p3k/p2Nnp2/r6P/6P1/2P3K1/4R3 w - - 1 46
Blunder: g5h6
Stockfish Evaluation: 1.71
Maia 1100 Evaluation: 1.29, Suggested Move: g2h3
Maia 1200 Evaluation: 1.0, Suggested Move: d5f6
Maia 1300 Evaluation: -0.04, Suggested Move: d5e7
Maia 1400 Evaluation: 1.19, Suggested Move: d5f6
Maia 1500 Evaluation: 1.48, Suggested Move: d5f6
Maia 1600 Evaluation: 1.72, Suggested Move: d5e7
Maia 1700 Evaluation: 2.61, Suggested Move: d5e7
Maia 1800 Evaluation: 2.73, Suggested Move: d5f6
Maia 1900 Evaluation: 2.45, Suggested Move: d5e7

Position: 8/7k/5R2/5p1P/r7/6PK/8/8 w - - 5 70
Blunder: c4a4
Stockfish Evaluation: 3.89
Maia 1100 Evaluation: 0.71, Suggested Move: f6f5
Maia 1200 Evaluation: 0.91, Suggested Move: f6f5
Maia 1300 Evaluation: 1.05, Suggested Move: f6f5
Maia 1400 Evaluation: 1.11, Suggested Move: f6f5
Maia 1500 Evaluation: 0.96, Suggested Move: f6f5
Maia 1600 Evaluation: 1.2, Suggested Move: f6f5
Maia 1700 Evaluation: 1.09, Suggested Move